# Build a Simple LLM Application with LangChain


[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/trgardos/ml-549-fa24/blob/main/src/langchain/build-simple-llm-app-with-langchain.ipynb)

> Adapted from [LangChain's Build a Simple LLM App](https://python.langchain.com/docs/tutorials/llm_chain/)
> and [llm_chain.ipynb](https://github.com/langchain-ai/langchain/blob/master/docs/docs/tutorials/llm_chain.ipynb).


In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After reading this tutorial, you'll have a high level overview of:

- Using [language models](https://python.langchain.com/docs/concepts/chat_models)

- Using [PromptTemplates](https://python.langchain.com/docs/concepts/prompt_templates) and [OutputParsers](https://python.langchain.com/docs/concepts/output_parsers)

- Using [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/concepts/lcel) to chain components together

- Debugging and tracing your application using [LangSmith](https://docs.smith.langchain.com/)

- Deploying your application with [LangServe](https://python.langchain.com/docs/concepts/#langserve)

Let's dive in!


## Setup

### Installation

To install LangChain, optionally create a python virtual environment, then run:

```bash
# from pip
pip install langchain langchain-openai
```

```bash
# from conda
conda install langchain langchain-openai -c conda-forge
```

For more details, see the [Installation guide](https://python.langchain.com/docs/how_to/installation/).

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install langchain langchain-openai

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

1. Go to [LangSmith](https://smith.langchain.com) to create an account or sign in.
1. After you create an account you will be sent a confirmation email. When you click
   on the link in the confirmation email you are taken to a page with a `Get Started` link.
1. Click on the `Get Started` link to go to the LangSmith setup page if this is your first time.

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
export LANGCHAIN_API_KEY="..."
export LANGCHAIN_PROJECT="pr-back-questionnaire-59"  # or whatever you want your project name to be
```

Or, if in a notebook, you can set them with:

In [ ]:
# Optional: You don't have to run this cell if you set the environment variables above

import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

import sys
if 'google.colab' in sys.modules:
    from google.colab import userdata
    os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY') if userdata.get('LANGCHAIN_API_KEY') else getpass.getpass("Enter your LangSmith API key:")
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') if userdata.get('OPENAI_API_KEY') else getpass.getpass("Enter your OpenAI API key:")
else:
    # prompt for your LangSmith API key
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter your LangSmith API key:") if not os.environ.get('LANGCHAIN_API_KEY') else os.environ.get('LANGCHAIN_API_KEY')
    # prompt for your OpenAI API key
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key:") if not os.environ.get('OPENAI_API_KEY') else os.environ.get('OPENAI_API_KEY')
    
os.environ["LANGCHAIN_PROJECT"] = "ds549-simple-llm-app" # or whatever you want your project name to be



To test your setup, you can run the following cell.

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")


AIMessage(content='Hello there! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f1a9935e-ad2f-446e-913f-2d5d9a9a7264-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

You should see a response similar to

```bash
AIMessage(content='Hello there! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f1a9935e-ad2f-446e-913f-2d5d9a9a7264-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})
```

Once this works for now, you can click `skip for now` to jump to your LangSmith home.

## Using Language Models

First up, let's learn how to use a language model by itself. 

LangChain supports many different language models that you can use interchangeably.
For now, we'll use OpenAI, but you can navigate to the [tutorial page](https://python.langchain.com/docs/tutorials/llm_chain/)
to try other APIs.

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [12]:
print(type(model))

<class 'langchain_openai.chat_models.base.ChatOpenAI'>


Let's first use the model directly. 

`ChatModel`s are instances of LangChain 
[`Runnables`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html),
which means they expose a standard interface for interacting with them. 

To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9b0a7b89-4d47-4250-a809-b81c0a53d137-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Theoretically, if you've enabled LangSmith, you should be able to see this run
logged to LangSmith, but at the time of this writing, it wasn't logging for this example.

There is a public example of the [LangSmith trace](https://smith.langchain.com/public/88baa0b2-7c1a-4d09-ba30-a47985dde2ea/r).

## OutputParsers


Notice that the response from the model is an `AIMessage`. 

This contains a string response along with other metadata about the response. 

Oftentimes we may just want to work with the string response. 

We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [14]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [15]:
result = model.invoke(messages)

In [16]:
parser.invoke(result)

'Ciao!'

More commonly, we can "chain" the model with this output parser. 

This means this output parser will get called every time in this chain. 

This chain takes on the input type of the language model (string or list of
message) and returns the output type of the output parser (string).

We can easily create the chain using the `|` operator. The `|` operator is used in LangChain to combine two elements together.

In [19]:
chain = model | parser

> This seems to be a clever override of the python `|` (or) operator which apparently
> is implemented as a `__or__` method in the `Runnable` base class.

In [20]:
chain.invoke(messages)

'Ciao!'

If LangSmith was working, we would see that the chain has two steps: 

1. first the language model is called, 
2. then the result of that is passed to the output parser. 

Here's a public exmaple of the [LangSmith trace]( https://smith.langchain.com/public/f1bdf656-2739-42f7-ac7f-0f1dd712322f/r)

### Get a Graph Representation of the Chain

You can see a graph representation of the chain.

In [ ]:
# grandalf is a pre-requisite
!pip install grandalf

In [24]:
chain.get_graph().print_ascii()

   +-----------------+     
   | ChatOpenAIInput |     
   +-----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


## Prompt Templates

Right now we are passing a list of messages directly into the language model. 

Where does this list of messages come from? 

Usually, it is constructed from a combination of user input and application logic. 

This application logic usually takes the raw user input and transforms it into a
list of messages ready to pass to the language model. 

Common transformations include adding a system message or formatting a template
with the user input.

PromptTemplates are a concept in LangChain designed to assist with this
transformation. 

They take in raw user input and return data (a prompt) that is ready to pass
into a language model. 

Let's create a PromptTemplate here. It will take in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate

In [26]:
from langchain_core.prompts import ChatPromptTemplate

First, let's create a string that we will format to be the system message:

In [27]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. 

This will be a combination of the `system_template` as well as a simpler
template for where to put the text to be translated

In [28]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [29]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [30]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

## Chaining together components with LCEL


We can now combine this with the model and the output parser from above using the pipe (`|`) operator:

In [31]:
chain = prompt_template | model | parser

In [32]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'

We can also print out the chain.

In [33]:
chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


This is a simple example of using
[LangChain Expression Language (LCEL)](https://python.langchain.com/docs/concepts/lcel/)
to chain together LangChain modules. 

There are several benefits to this approach, including optimized streaming and tracing support.

If we take a look at the LangSmith trace, we can see all three components show up.

Here's the public example in [LangSmith trace](https://smith.langchain.com/public/bc49bec0-6b13-4726-967f-dbd3448b786d/r).

## Conclusion

For further reading on the core concepts of LangChain, see [Conceptual Guides](https://python.langchain.com/docs/concepts).

See more detailed guides:

- [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/how_to/#langchain-expression-language-lcel)
- [Prompt templates](https://python.langchain.com/docs/how_to/#prompt-templates)
- [Chat models](https://python.langchain.com/docs/how_to/#chat-models)
- [Output parsers](https://python.langchain.com/docs/how_to/#output-parsers)
- [LangServe](https://python.langchain.com/docs/langserve/)

And the LangSmith docs:

- [LangSmith](https://docs.smith.langchain.com)